In [2]:
%load_ext tensorboard

In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorboard.plugins.hparams import api as hp

In [3]:
import pathlib
import os.path

data_dir =  pathlib.Path(os.path.abspath("images/Flowers5/"))
data_dir

WindowsPath('E:/school/UCSD-ML-Capstone/images/Flowers5')

In [4]:
batch_size = 32
epochs=25
img_height = 192
img_width = 192

In [5]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(data_dir, validation_split=0.2, subset="both", seed=123,
                                                               image_size=(img_height, img_width), batch_size=batch_size)

Found 1857 files belonging to 5 classes.
Using 1486 files for training.
Using 371 files for validation.


In [6]:
class_names = train_ds.class_names
num_classes = len(class_names)
num_classes
print(class_names)

['Calceolaria', 'Daffodil', 'Freesia', 'Hibiscus', 'Iris']


In [7]:
model_url = 'https://tfhub.dev/google/bit/s-r101x1/1'
base_model = hub.KerasLayer(model_url)

In [8]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 48]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.8))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_F1_SCORE = 'f1_score'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_F1_SCORE, display_name='F1 Score')],
    )

In [9]:
from sklearn.metrics import f1_score as sk_f1

# Define our own function since we are working with batch datasets.
def f1_score(model):
    y_pred = []  # store predicted labels
    y_true = []  # store true labels

    # We iterate over the dataset to get the true label and input at the same time.
    # Otherwise we will unintentionally shuffle the dataset.
    for image_batch, label_batch in val_ds:
        y_true.append(label_batch)
        preds = model.predict(image_batch, verbose=0)
        y_pred.append(np.argmax(preds, axis = - 1))

    # convert the true and predicted labels into tensors
    correct_labels = tf.concat([item for item in y_true], axis = 0)
    predicted_labels = tf.concat([item for item in y_pred], axis = 0)
    
    return sk_f1(correct_labels, predicted_labels, average=None)

In [10]:
def train_test_model(hparams):
    model = tf.keras.Sequential([
        layers.Rescaling(1./255),
        base_model,
        layers.Flatten(),
        layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
        layers.Dropout(hparams[HP_DROPOUT]),
        layers.Dense(num_classes),
    ])
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=1)
    return np.mean(f1_score(model))

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        f1 = train_test_model(hparams)
        tf.summary.scalar(METRIC_F1_SCORE, f1, step=1)

In [12]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in np.linspace(
        HP_DROPOUT.domain.min_value,
        HP_DROPOUT.domain.max_value,
        num=3,
    ):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
Epoch 1/25
47/47 [==============================] - 36s 311ms/step - loss: 1.0648 - accuracy: 0.5895 - val_loss: 0.6167 - val_accuracy: 0.8302
Epoch 2/25
47/47 [==============================] - 12s 249ms/step - loss: 0.6583 - accuracy: 0.7470 - val_loss: 0.4860 - val_accuracy: 0.8464
Epoch 3/25
47/47 [==============================] - 12s 246ms/step - loss: 0.5295 - accuracy: 0.8062 - val_loss: 0.4195 - val_accuracy: 0.8868
Epoch 4/25
47/47 [==============================] - 12s 246ms/step - loss: 0.4713 - accuracy: 0.8277 - val_loss: 0.3569 - val_accuracy: 0.8949
Epoch 5/25
47/47 [==============================] - 12s 246ms/step - loss: 0.4007 - accuracy: 0.8614 - val_loss: 0.3097 - val_accuracy: 0.9191
Epoch 6/25
47/47 [==============================] - 12s 246ms/step - loss: 0.3800 - accuracy: 0.8668 - val_loss: 0.2965 - val_accuracy: 0.9137
Epoch 7/25
47/47 [==============================] - 12s 246ms

In [9]:
%tensorboard --logdir logs/hparam_tuning --host localhost